In [ ]:
import numpy as np
import cv2
import glob
import ntpath
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random

objpoints = []
imgpoints = []
nx = 9
ny = 6

def camera_calibrate():
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2]=np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    images = glob.glob('./camera_cal/*.jpg')
    i = 0
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            i = i + 1
            print(i)
            img = cv2.drawChessboardCorners(img, (nx,ny), corners,ret)
            plt.figure(i)
            plt.imshow(img)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    return mtx, dist

In [ ]:
#Calibrate the Camera
mtx,dist = camera_calibrate()

In [ ]:
#Randomly select a chessboard pattern and undistort it
images = glob.glob('./camera_cal/*.jpg')
index = random.randint(0, len(images))
print(images[index])
img = cv2.imread(images[index])
undist = cv2.undistort(img, mtx, dist, None, mtx)
plt.imshow(undist)
cv2.imwrite('./undist.jpg',undist)

In [ ]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel= cv2.Sobel(gray,cv2.CV_64F, 1, 0) 
    elif orient == 'y':
        sobel= cv2.Sobel(gray,cv2.CV_64F, 0, 1)
        
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1, ksize = sobel_kernel)
    # 3) Calculate the magnitude 
    mag = np.sqrt((np.square(sobelx)+np.square(sobely)))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*mag/np.max(mag))
    # 5) Create a binary mask where mag thresholds are met
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(grad)
    dir_binary[(grad >= thresh[0]) & (grad <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return dir_binary

In [ ]:
def rgb_r_select(img, thresh=(0, 255)):
    rgb_r = img[:,:,0]
    binary_output = np.zeros_like(rgb_r)
    binary_output[(rgb_r > thresh[0]) & (rgb_r <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

In [ ]:
def rgb_g_select(img, thresh=(0, 255)):
    rgb_g = img[:,:,1]
    binary_output = np.zeros_like(rgb_g)
    binary_output[(rgb_g > thresh[0]) & (rgb_g <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

In [ ]:
def hls_s_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    hls_s = hls[:,:,2]
    binary_output = np.zeros_like(hls_s)
    binary_output[(hls_s > thresh[0]) & (hls_s <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

In [ ]:
def hls_l_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    hls_l = hls[:,:,1]
    binary_output = np.zeros_like(hls_l)
    binary_output[(hls_l > thresh[0]) & (hls_l <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

In [ ]:
#Undistort Test images
test_images = glob.glob('./test_images/*.jpg')
for test_image in test_images:
    base = ntpath.basename(test_image)
    image = cv2.imread(test_image)
    image = cv2.undistort(image, mtx, dist, None, mtx)
    filename = './test_images/undistort_'+base
    cv2.imwrite(filename, image)

In [ ]:
#Apply Color and gradient thresholding
ksize=9
test_images = glob.glob('./test_images/undistort_*.jpg')
for i,test_image in enumerate(test_images):
    base = ntpath.basename(test_image)
    image = cv2.imread(test_image)
    imagergb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rgb_binary = rgb_r_select(imagergb, thresh=(200,255))
    binary = hls_s_select(imagergb, thresh=(120,255))
    gradx = abs_sobel_thresh(imagergb, orient='x', sobel_kernel=ksize, thresh=(40, 100))
    combined = np.zeros_like(binary)
    combined[((binary == 1) & (rgb_binary == 1))] = 255
    filename = './test_images/hr_'+base
    cv2.imwrite(filename, combined)
    print(i)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(imagergb)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(combined, cmap='gray')
    ax2.set_title('Thresholded S', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
#Select the area of road that is to be used
#as source points for perspective transform
src = np.float32([[685,445],
          [1052,673],
          [256,673],
          [595,445]])
#Select the image coordinates that is to be used
#as destination points for perspective transform
dst =np.float32([[950,0],
          [950,720],
          [300,720],
          [300,0]])

#Get Perspective transform Matrix and
#Inverse PErspective transform Matrix
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

In [ ]:
def find_curvature(lefty, righty, leftx_pix, rightx_pix, ploty):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/650 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    # Fit new polynomials to x,y in world space
    leftx = np.copy(leftx_pix)
    rightx = np.copy(rightx_pix)
    leftx=leftx[::-1]
    rightx=rightx[::-1]
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad,right_curverad

In [ ]:
frames = 0
left_fit = np.zeros((3,), dtype=np.float64)
right_fit = np.zeros((3,), dtype=np.float64)
def detect_line(binary_warped, search_entire_frame=0):
    global frames
    global left_fit
    global right_fit

    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    out_img = out_img.astype(np.uint8)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    #Use higly targeted search based on the lane information from
    #previous frames
    if (frames != 0) and (search_entire_frame == 0):
        print("use previous frame")
        left_lane_inds = []
        right_lane_inds = []
        margin = 100
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))
        if (len(left_lane_inds) < 1000 or len(right_lane_inds) < 1000):
            search_entire_frame = 1
    
    #Search lanes pixels using histogram peak method
    if (frames == 0) or (search_entire_frame == 1):
        print("enitre frame")
        # Take a histogram of the bottom half of the warped binary image
        histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
        
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        
        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

    frames = frames + 1
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
        
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
        
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    #Calculate the car offset from the lane center
    xm_per_pix = 3.7/650 # meters per pixel in x dimension
    center_lane = (left_fitx[-1] + right_fitx[-1])/2
    car_center = binary_warped.shape[1]/2
    car_offset = (center_lane - car_center)*xm_per_pix
    
    left_curverad,right_curverad = find_curvature(lefty, righty, leftx, rightx, ploty)
    
    #Calculate average radius of curvature
    curvature = (left_curverad+right_curverad)/2
    
    #Color the lanes
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    return out_img,left_fitx,right_fitx,ploty, curvature, car_offset

In [ ]:
def draw_lane_lines(undist, Minv, left_fitx, right_fitx, ploty):
    # Create an image to draw the lines on
    warp_zero = np.zeros(((undist.shape[0], undist.shape[1]))).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

In [ ]:
#Experiment sobel and hsl for getting peaks
#Need to check R select to get a robust as it is detecting cars as well
ksize=9
test_images = glob.glob('./test_images/undistort_*.jpg')
for i,test_image in enumerate(test_images):
    base = ntpath.basename(test_image)
    image = cv2.imread(test_image)
    img_size=(image.shape[1],image.shape[0])
    imagergb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gradx = abs_sobel_thresh(imagergb, orient='x', sobel_kernel=ksize, thresh=(40, 100))
    hls_binary = hls_s_select(imagergb, thresh=(120,255))
    rgb_binary = rgb_r_select(imagergb, thresh=(200,255))
    combined = np.zeros_like(rgb_binary)

    combined[(gradx == 1) | ((hls_binary == 1) & (rgb_binary == 1))] = 255
    warped = cv2.warpPerspective(combined, M, img_size, flags=cv2.INTER_LINEAR)
    histogram = np.sum(warped[int(warped.shape[0]/2):,:], axis=0)
    line_image, left_x, right_x, y_coord, curvature, car_offset = detect_line(warped, 1)
    final = draw_lane_lines(imagergb, Minv, left_x, right_x, y_coord)
    text_l = "Curvature ="+str(curvature)
    text_r = "Car offset ="+str(car_offset)
    cv2.putText(final,text_l, (700,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)
    cv2.putText(final,text_r, (700,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(combined, cmap='gray')
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(line_image)
    ax2.plot(left_x, y_coord, color='yellow')
    ax2.plot(right_x, y_coord, color='yellow')
    ax2.set_title('Detected Lines', fontsize=50)
    ax3.plot(histogram)
    ax3.set_title('Histogram', fontsize=50)
    ax4.imshow(final)
    ax4.set_title('Unwarped Lane Lines', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
test_images = glob.glob('./test_images/undistort_*.jpg')
for i,test_image in enumerate(test_images):
    image = cv2.imread(test_image)
    imagergb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(imagergb)
    plt.plot(685,445,'.')
    plt.plot(1052,673,'.')
    plt.plot(256,673,'.')
    plt.plot(595,445,'.')

In [ ]:
from collections import deque
i=0
left_data=deque(maxlen=10)
right_data=deque(maxlen=10)
def process_image(image):
    global mtx
    global dist
    global i
    ksize=9
    img_size=(image.shape[1],image.shape[0])
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    imagergb = np.copy(undist)
    gradx = abs_sobel_thresh(imagergb, orient='x', sobel_kernel=ksize, thresh=(40, 100))
    hls_binary = hls_s_select(imagergb, thresh=(120,255))
    rgb_binary = rgb_r_select(imagergb, thresh=(200,255))
    combined = np.zeros_like(rgb_binary)
    combined[(gradx == 1) | ((hls_binary == 1) & (rgb_binary == 1))] = 255
    warped = cv2.warpPerspective(combined, M, img_size, flags=cv2.INTER_LINEAR)
    line_image, left_x, right_x, y_coord, curvature, car_offset = detect_line(warped, 0)
    #Store upto lleft lane and right lane coordinates
    left_data.append(left_x)
    right_data.append(right_x)
    if i >= 9:
        #Weighted average with the last 9 images
        left_lane_array = np.array(left_data)
        right_lane_array = np.array(right_data)
        left_x_avg = np.average(left_lane_array, axis=0, weights=range(1,11))
        right_x_avg = np.average(right_lane_array, axis=0, weights=range(1,11))
    else:
        left_x_avg = np.copy(left_x)
        right_x_avg = np.copy(right_x)
        
    final = draw_lane_lines(imagergb, Minv, left_x_avg, right_x_avg, y_coord)
    text_l = "Curvature ="+str(curvature)
    text_r = "Car Offset ="+str(car_offset)
    cv2.putText(final,text_l, (700,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)
    cv2.putText(final,text_r, (700,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)
    imagebgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    finalbgr = cv2.cvtColor(final, cv2.COLOR_RGB2BGR)
    filename = './project_images/image_'+str(i)+".jpg"
    cv2.imwrite(filename, imagebgr)
    i=i+1
    return final

In [ ]:
frames = 0
left_fit = np.zeros((3,), dtype=np.float64)
right_fit = np.zeros((3,), dtype=np.float64)
video = VideoFileClip('project_video.mp4')
video_output='out_project_final.mp4'
output_clip = video.fl_image(process_image)
output_clip.write_videofile(video_output, audio=False)